In [1]:
import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
class Stock:
    def __init__(self,name,start,end):
        self.name = name
        self.df = web.DataReader(self.name, 'yahoo', start, end)
        self.vals = self.df['Adj Close'].values
        self.time = self.df.index
        self.valsNorm = self.vals/self.vals[0]
class Stocks:
    def __init__(
        self,
        *args,
        start = datetime.now() - relativedelta(years=1),
        end = datetime.now()):
        self.listOfStocks = []
        for arg in args:
            self.listOfStocks.append(Stock(arg,start=start,end=end))

In [3]:
# Set start and end as one year ago to now
endDate = datetime.now()
startDate = endDate - relativedelta(years=1)
# Create stocks object
stocks = Stocks('MSFT','AAPL','GOOGL',start=startDate,end=endDate)

In [4]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected = True)

data = []
for stock in stocks.listOfStocks:
    data.append(
        go.Scatter(
            x = stock.time,
            y = stock.valsNorm,
            name = stock.name
        )
    )
layout = go.Layout(
  title = 'stocks',
  yaxis = dict(
    hoverformat = '.3g'
  )
)
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)